# CNN分类
导入所需库

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

定义CNN模型

In [39]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=17):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 28 * 28, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


图片格式转化并加载数据

In [40]:


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


dataset = ImageFolder(root='E:\\code\\Jupyter\\final_repo\\pics', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
class_to_idx = dataset.class_to_idx
print(class_to_idx)

{'Analgesic': 0, 'Anti-inflammatory': 1, 'Antibacterial': 2, 'Antidepressant': 3, 'Antidiabetic': 4, 'Antifungal': 5, 'Antihistamine': 6, 'Antihypertensive': 7, 'Antioxidant': 8, 'Antiprotozoal': 9, 'Antipyretic': 10, 'Antispasmodic': 11, 'Antitumor': 12, 'Antiviral': 13, 'Diuretic': 14, 'Hypnotic': 15, 'Sedative': 16}


初始化模型

In [41]:

model = SimpleCNN(num_classes=17)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_acc = correct / total * 100
        train_loss = running_loss / len(train_loader)
        val_acc = evaluate(model, val_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, Val Accuracy: {val_acc:.2f}%')
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_cnn_model2.pth')
    print(f"Training completed. Best validation accuracy: {best_val_acc:.2f}%")
# 评估函数
def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total * 100


cuda


训练模型  
(之前训练过一轮，有图片证明，模型保存至CNN_Model.pth,不小心重启了一次)

In [42]:
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=30)

Epoch [1/30], Train Loss: 2.9968, Train Accuracy: 13.23%, Val Accuracy: 20.61%
Epoch [2/30], Train Loss: 2.4048, Train Accuracy: 21.45%, Val Accuracy: 28.33%


KeyboardInterrupt: 

测试测试集

In [6]:
# Load the best model and evaluate on validation set
model.load_state_dict(torch.load(r'E:\code\Jupyter\final_repo\CNN.pth'))
val_acc = evaluate(model, val_loader)
print(f"Final Validation Accuracy: {val_acc:.2f}%")

C:\Users\lenovo\AppData\Local\Temp\ipykernel_26916\1579089669.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_cnn_model.pth'))


Final Validation Accuracy: 43.23%


测试计算某药性的所有图片准确率

In [50]:
import os
from PIL import Image
pre_test_folder = r'E:\code\Jupyter\final_repo\pics'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# 加载模型权重
model.load_state_dict(torch.load(r'E:\code\Jupyter\final_repo\CNN_Model.pth', map_location=device))
model = model.to(device)
model.eval()
count = 0
lens = 0
classes = {'Analgesic': 0, 'Anti-inflammatory': 1, 'Antibacterial': 2, 'Antidepressant': 3, 'Antidiabetic': 4, 'Antifungal': 5, 'Antihistamine': 6, 'Antihypertensive': 7, 'Antioxidant': 8, 'Antiprotozoal': 9, 'Antipyretic': 10, 'Antispasmodic': 11, 'Antitumor': 12, 'Antiviral': 13, 'Diuretic': 14, 'Hypnotic': 15, 'Sedative': 16}
clas = 0
for i in classes:
    test_folder = pre_test_folder + '\\' + i    
    # 遍历文件夹中的所有图片
    for filename in os.listdir(test_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(test_folder, filename)
            image = Image.open(img_path).convert('RGB')  # 确保图像是RGB格式
            image = transform(image).unsqueeze(0).to(device)
            
            # 预测
            with torch.no_grad():
                output = model(image)
                _, predicted = torch.max(output, 1)
            print(f'Image: {filename}, Predicted Class: {predicted.item()},progress: {lens}')
            lens += 1
            if clas == predicted.item():
                count += 1
    print(f'{i} Accuracy: {count/lens}')
    clas += 1
    lens = 0
    count = 0

cuda


C:\Users\lenovo\AppData\Local\Temp\ipykernel_36504\1670593622.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r'E:\code\Jupyter\final_re

Image: CID_10010848.png, Predicted Class: 0,progress: 0
Image: CID_10041933.png, Predicted Class: 0,progress: 1
Image: CID_100472.png, Predicted Class: 0,progress: 2
Image: CID_10055958.png, Predicted Class: 0,progress: 3
Image: CID_10070040.png, Predicted Class: 0,progress: 4
Image: CID_10079.png, Predicted Class: 0,progress: 5
Image: CID_10080.png, Predicted Class: 3,progress: 6
Image: CID_10090.png, Predicted Class: 0,progress: 7
Image: CID_100962152.png, Predicted Class: 1,progress: 8
Image: CID_10100.png, Predicted Class: 0,progress: 9
Image: CID_10101.png, Predicted Class: 0,progress: 10
Image: CID_101243415.png, Predicted Class: 13,progress: 11
Image: CID_10151715.png, Predicted Class: 1,progress: 12
Image: CID_10154.png, Predicted Class: 8,progress: 13
Image: CID_101599248.png, Predicted Class: 0,progress: 14
Image: CID_101603078.png, Predicted Class: 1,progress: 15
Image: CID_101611440.png, Predicted Class: 0,progress: 16
Image: CID_10170.png, Predicted Class: 1,progress: 17
I

KeyboardInterrupt: 

好的，我们将使用之前在 `CNN.ipynb` 中训练的模型来测试所有数据的准确率，并将结果绘制成热力图。以下是完整的代码实现：

### 1. 导入所需库


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
from PIL import Image
import os



### 2. 数据加载函数


In [ ]:
def data_load(data_dir, img_height, img_width, batch_size, num_workers=8):
    transform = transforms.Compose([
        transforms.Resize((img_height, img_width)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    dataset = ImageFolder(root=data_dir, transform=transform)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    
    class_names = dataset.classes
    return loader, class_names



### 3. 定义CNN模型


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 28 * 28, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x



### 4. 测试函数


In [ ]:
def test_model(model, test_loader, device):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    return y_true, y_pred



### 5. 绘制混淆矩阵


In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names):
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix, index=[i for i in class_names], columns=[i for i in class_names])
    plt.figure(figsize=(12, 7))
    sns.heatmap(df_cm, annot=True, fmt='g')
    plt.savefig('cnn_confusion_matrix.png', dpi=100)



### 6. 主函数


In [ ]:
if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    img_size = 224
    batch_size = 32
    test_data_dir = "fruit30_split/test"  # 测试数据集路径
    test_loader, class_names = data_load(test_data_dir, img_size, img_size, batch_size)
   
    model = SimpleCNN(num_classes=len(class_names)).to(device)
    model.load_state_dict(torch.load("simple_cnn.pth"))  # 加载之前训练的模型权重
    
    y_true, y_pred = test_model(model, test_loader, device)
    
    test_accuracy = np.sum(np.array(y_true) == np.array(y_pred)) / len(y_true)
    print(f"Test Accuracy: {test_accuracy:.2f}%")
    plot_confusion_matrix(y_true, y_pred, class_names)



通过以上代码，我们加载了之前训练的CNN模型，并使用该模型对测试数据集进行预测，计算准确率并绘制混淆矩阵。请确保数据集路径和模型权重文件路径正确。

找到具有 2 个许可证类型的类似代码

测试单一图片的可能性

In [45]:
import torch.nn.functional as F
classes = {'Analgesic': 0, 'Anti-inflammatory': 1, 'Antibacterial': 2, 'Antidepressant': 3, 'Antidiabetic': 4, 'Antifungal': 5, 'Antihistamine': 6, 'Antihypertensive': 7, 'Antioxidant': 8, 'Antiprotozoal': 9, 'Antipyretic': 10, 'Antispasmodic': 11, 'Antitumor': 12, 'Antiviral': 13, 'Diuretic': 14, 'Hypnotic': 15, 'Sedative': 16}
# 进行预测
model.load_state_dict(torch.load('CNN_Model.pth', map_location=device))
def predict(image_path):
    image = load_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)  # 使用 softmax 获取概率分布
        _, predicted = torch.max(outputs, 1)
        return predicted.item(), probabilities.cpu().numpy()

# 示例：输入图片路径进行预测
image_path = r'E:\code\Jupyter\final_repo\pics\Analgesic\CID_904.png'
predicted_class, probabilities = predict(image_path)
print(f'Predicted class: {predicted_class}')

# 获取标签名称
class_names = dataset.classes
print(f'Predicted class name: {class_names[predicted_class]}')

# 打印概率分布
for i, prob in enumerate(probabilities[0]):
    print(f'Class: {class_names[i]}, Probability: {prob:.4f}')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_36504\330104571.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('CNN_Model.pth', map_locati

Predicted class: 0
Predicted class name: Analgesic
Class: Analgesic, Probability: 0.6186
Class: Anti-inflammatory, Probability: 0.1782
Class: Antibacterial, Probability: 0.0101
Class: Antidepressant, Probability: 0.0186
Class: Antidiabetic, Probability: 0.0002
Class: Antifungal, Probability: 0.0074
Class: Antihistamine, Probability: 0.0145
Class: Antihypertensive, Probability: 0.0008
Class: Antioxidant, Probability: 0.0926
Class: Antiprotozoal, Probability: 0.0027
Class: Antipyretic, Probability: 0.0391
Class: Antispasmodic, Probability: 0.0021
Class: Antitumor, Probability: 0.0007
Class: Antiviral, Probability: 0.0055
Class: Diuretic, Probability: 0.0019
Class: Hypnotic, Probability: 0.0033
Class: Sedative, Probability: 0.0037
